# Only Flask Server

In [1]:
from flask import Flask, request, jsonify
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
import numpy as np
import pandas as pd
import joblib
from flask_cors import CORS

loaded_pipeline = joblib.load('svm_model.joblib')

scaler = loaded_pipeline.named_steps['standardscaler']
svm_classifier = loaded_pipeline.named_steps['svc']

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

app = Flask(__name__)
CORS(app)

limiter = Limiter(
    get_remote_address,
    app=app,
    default_limits=["200 per day", "5 per hour"],
    storage_uri="memory://",
)

@app.route('/predict', methods=['POST'])
@limiter.limit("5 per minute")
def predict():
    data = request.get_json()
    x_test = data['features']
    received_array = np.array(x_test)
    columns_to_select = ['Attention', 'Mediation', 'Raw', 'Delta', 'Theta', 'Alpha1', 'Alpha2',
       'Beta1', 'Beta2', 'Gamma1', 'Gamma2', 'age', 'ethnicity', 'gender']
    received_dataframe = pd.DataFrame(data=[received_array], columns=columns_to_select)
    X_new = received_dataframe
    X_new_standardized = scaler.transform(X_new)
    prediction = svm_classifier.predict(X_new_standardized).astype(int)[0]

    confidence_scores = svm_classifier.decision_function(X_new_standardized)
    confidence_percentage = round(float(sigmoid(confidence_scores) * 100),2)

    return jsonify({'prediction': prediction.tolist(), 'confidence': confidence_percentage})


if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Feb/2024 15:18:28] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:18:29] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:20:23] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:20:23] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:20:34] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:20:35] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:22:23] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:22:23] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:22:49] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:22:49] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:23:02] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:23:03] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:24:23] "OPTIONS /predict HTTP/1.1" 200 -
127.0.0.1 - - [04/Feb/2024 15:24:23] "